<a href="https://colab.research.google.com/github/evlko/CS-388/blob/main/NLP_Lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LABORATY WORK 1

**Source link**

In [ ]:
DATA_URL = "http://lib.ru/LITRA/PUSHKIN/dubrowskij.txt"

Install libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')
!pip install -q --upgrade pip
!pip install -q keras==2.2.4
!pip install -q 'h5py<3.0.0'
!pip install -q -U numpy==1.18.5
!pip install -q tensorflow==1.15.2
!pip install -q PyYaml==5.3.1
!pip install -q rnnmorph==0.4.0

Initialize a morphological analyzer object `RNNMorph`

In [ ]:
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")

Download the text for the task using `urllib`

In [ ]:
import urllib.request

opener = urllib.request.URLopener({})
resource = opener.open(DATA_URL)
raw_text = resource.read().decode(resource.headers.get_content_charset())

Raw Text:

In [ ]:
raw_text[:200]

'<html><head><title>Александр Сергеевич Пушкин. Дубровский</title></head><body><pre><div align=right><form action=/LITRA/PUSHKIN/dubrowskij.txt><select name=format><OPTION VALUE="_Contents">Содержание<'

The text contains html tags which should be removed. Stripping HTML tags from text using the `Beatiful soap` lib

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(raw_text, features="html.parser")

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
cleaned_text = soup.get_text()

Cleand Text:

In [ ]:
cleaned_text[:200]

'Александр Сергеевич Пушкин. ДубровскийСодержаниеFine HTMLPrinted versiontxt(Word,КПК)Lib.ru html\nАлександр Сергеевич Пушкин. Дубровский\n\n---------------------------------------------------------------'

Using [NLTK](https://nltk.org/) lib we strip the text into sentences and tokens.

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

tokenized_sentences = [word_tokenize(sentence) for sentence in sent_tokenize(cleaned_text)]
"A total of %d 'sentences'" % len(tokenized_sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


"A total of 1426 'sentences'"

## Task 1
Using `str.isalpha` function from the Python standard library only literal tokens remain in predictions.

In [ ]:
from tqdm import tqdm
predictions = [[pred.normal_form for pred in sent if str(pred.normal_form).isalpha()] 
               for sent in tqdm(predictor.predict_sentences(sentences=tokenized_sentences), "sentences")]

sentences: 100%|██████████| 1426/1426 [00:00<00:00, 58722.64it/s]


Total Sentences:

In [ ]:
len(predictions)

1426

Total Tokens:

In [ ]:
non_uniq_tokens = [word for sentence in predictions for word in sentence]
len(non_uniq_tokens) 

20410

## Task 2

Using `non_uniq_tokens`, russian stop words from nltk (`nltk.corpus.stopwords`) lib and `nltk.FreqDist`, calculate, **what proportion of the 50 most frequent** tokens in the product is occupied by tokens **not related** to stop words.

**For example**, if among the 100 most frequent words there are 25 words included in the stop list, then 75 words are not included in the stop list, and their share will be 0.75.

In [ ]:
import nltk
from nltk import FreqDist
from nltk.corpus import stopwords
nltk.download("stopwords")
STOPWORDS = set(stopwords.words("russian"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Stop words examples:

In [ ]:
list(STOPWORDS)[:5] 

['без', 'кто', 'куда', 'сейчас', 'впрочем']

In [ ]:
from itertools import islice

def take(n, iterable):
    return list(islice(iterable, n))

fdist = FreqDist(non_uniq_tokens)

n = 50
words = fdist.most_common()[:n]
s = 0

for word in words:
  if word[0] not in STOPWORDS:
    s += 1

s / n

0.36

## Task 3
Calculate how many tokens occur in the text **strictly more than** 20 times.

In [ ]:
words = fdist.most_common()
c = 0

for word in words:
  if word[1] > 20:
    c += 1
  else:
    break

c

131